DSPy is a library that changes the way we do prompting, in DSPy instead of writing prompts, we can do programming and DSPy will take care of prompts. It majorly has 3 components: **Signatures, Modules and Teleprompters**

## Install the library

In [1]:
!pip3 install datasets dspy-ai

In this notebook, we will see an example of using DSPy for the task of Text2SQL, we will see how DSPy saves our time in writing handcrafted prompts

In [2]:
#import necessary libraries
import dspy
import os 
import pandas as pd
from datasets import load_dataset
from dspy.teleprompt import *

turbo = dspy.OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    model_type="chat"
)

#set the language model
dspy.configure(lm=turbo)

In [27]:
sample_question = """question : How much money has Candidate Dennis Kucinich spent? 
 Table Headers : Candidate,Money Raised, 3Q,Loans Received, 3Q,Money Spent, 3Q,Total Receipts,Cash On Hand,After Debt 
 SQL query : """

DSPy's fundamental module is Signature <br> <br>
1.A Signature consists of a simple description of the task(this will be later used to create prompt internally by DSPy). <br>2. A description of one or more Input field and a description of one or more output fields

In [28]:
class BasicText2SQL(dspy.Signature):
    """Convert the given question along with the table headers to SQL query"""
    question = dspy.InputField(desc="question with the table headers")
    answer = dspy.OutputField(desc="SQL query")

#define the predictor
query_generator = dspy.Predict(BasicText2SQL)
query = query_generator(question=sample_question)

In [29]:
print(query.answer)

SELECT "Money Spent, 3Q" FROM table_name WHERE Candidate = 'Dennis Kucinich'


We can notice how we got the query with minimal prompt writing. Let's use some data and see how we can increase the efficieny

In [3]:
#load the wikkisql data
wikisql_data = load_dataset("wikisql")

In [4]:
def fetch_table_context(table):
    """
    Fetch the column names
    """
    header = [f"{header}" for header in table['header']]
    return ",".join(header)


def preprocess_function(examples):
    """
    Fetches the questions
    """
    text_column = "question"
    inputs = [f"{text_column} : {x} \n Table Headers : {fetch_table_context(t)} \n SQL query : " for x,t in zip(examples[text_column],examples['table'])] 
    examples['question'] = inputs
    examples['answer'] = examples['sql']
    return examples

In [5]:
train_processed_datasets = wikisql_data['train'].map(
    preprocess_function,
    batched=True,
    num_proc=1,
    load_from_cache_file=False,
    desc="Processing the Dataset",
)

Processing the Dataset: 100%|██████████| 56355/56355 [00:06<00:00, 8551.63 examples/s]


In [6]:
wikisql_train_df = train_processed_datasets.to_pandas()
#select 1000 for train, 500 for dev 
wikisql_train = wikisql_train_df.sample(1000)
wikisql_dev = wikisql_train_df.sample(500)

In [7]:
wikisql_train

,phase,question,table,sql,answer
33674,2,question : What is the elista with 1 played an...,"{'header': ['Player', 'Qual.', 'Baku', 'Sochi'...",{'human_readable': 'SELECT Elista FROM table W...,"{'agg': 0, 'conds': {'column_index': [8, 2], '..."
1454,1,question : Who did the high points of game 5? ...,"{'header': ['Game', 'Date', 'Team', 'Score', '...",{'human_readable': 'SELECT High points FROM ta...,"{'agg': 0, 'conds': {'column_index': [0], 'con..."
15050,1,question : What is the date of the finale wher...,"{'header': ['Series', 'Premiere', 'Finale', 'R...",{'human_readable': 'SELECT Finale FROM table W...,"{'agg': 0, 'conds': {'column_index': [3], 'con..."
19253,2,question : What score has nashville predators ...,"{'header': ['Game', 'December', 'Opponent', 'S...",{'human_readable': 'SELECT Score FROM table WH...,"{'agg': 0, 'conds': {'column_index': [2], 'con..."
39550,2,question : I want the constellation for declin...,"{'header': ['NGC number', 'Object type', 'Cons...",{'human_readable': 'SELECT Constellation FROM ...,"{'agg': 0, 'conds': {'column_index': [4, 3], '..."
...,...,...,...,...,...
729,1,question : On which episode did actress Sela W...,"{'header': ['Character', 'Portrayed by', 'Firs...",{'human_readable': 'SELECT Last appearance FRO...,"{'agg': 0, 'conds': {'column_index': [1], 'con..."
35759,2,question : What was the date when the away tea...,"{'header': ['Home team', 'Home team score', 'A...",{'human_readable': 'SELECT Date FROM table WHE...,"{'agg': 0, 'conds': {'column_index': [2], 'con..."
31992,2,question : What is the Date of the match with ...,"{'header': ['Date', 'Tournament', 'Surface', '...",{'human_readable': 'SELECT Date FROM table WHE...,"{'agg': 0, 'conds': {'column_index': [3], 'con..."
37367,2,question : What is the density of San Sebastiá...,"{'header': ['City district', 'Area km²', 'Popu...",{'human_readable': 'SELECT Density (hab/km²) F...,"{'agg': 0, 'conds': {'column_index': [0], 'con..."


In [8]:
wikisql_train['answer'] = wikisql_train['answer'].map(lambda x: x['human_readable'])
wikisql_dev['answer'] = wikisql_dev['answer'].map(lambda x: x['human_readable'])

In [10]:
del wikisql_train['sql'] 
del wikisql_train['table']
del wikisql_train['phase']

del wikisql_dev['sql'] 
del wikisql_dev['table']
del wikisql_dev['phase']

In [11]:
train_df_records  = wikisql_train.to_dict('records')
test_df_records  = wikisql_dev.to_dict('records')


# Create dataset
train_dataset = [dspy.Example(x).with_inputs('question') for x in train_df_records]
test_dataset = [dspy.Example(x).with_inputs('question') for x in test_df_records]


There are multiple techniques in Prompting such as ChainOfThought, ReACT, Tree of Thoughts, etc.. so if I want to implement those in DSPy, should I write all the complete prompts?, No. DSPy has methods integrating those prompt techniques, we can directly use that

In [32]:
generate_query_with_cot = dspy.ChainOfThought(BasicText2SQL)
cot_query = generate_query_with_cot(question=test_dataset[0].question)

In [33]:
cot_query

Prediction(
    rationale='produce the answer. We need to find the row for Candidate Dennis Kucinich and retrieve the value in the "Money Spent, 3Q" column.',
    answer='SQL query: SELECT "Money Spent, 3Q" FROM table_name WHERE Candidate = \'Dennis Kucinich\''
)

In [34]:
test_dataset[0].answer

'SELECT Money Spent, 3Q FROM table WHERE Candidate = dennis kucinich'

We can check the prompt by here

In [35]:
turbo.inspect_history()





Convert the given question along with the table headers to SQL query

---

Follow the following format.

Question: question with the table headers
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: SQL query

---

Question: question : How much money has Candidate Dennis Kucinich spent? Table Headers : Candidate,Money Raised, 3Q,Loans Received, 3Q,Money Spent, 3Q,Total Receipts,Cash On Hand,After Debt SQL query :
Reasoning: Let's think step by step in order to produce the answer. We need to find the row for Candidate Dennis Kucinich and retrieve the value in the "Money Spent, 3Q" column.
Answer: SQL query: SELECT "Money Spent, 3Q" FROM table_name WHERE Candidate = 'Dennis Kucinich'





## Module

Modules are another fundamental building blocks of DSPy, Modules helps us to enable many features of DSPy such as teleprompters, also modules helps us to use the prompts and classes which will further increases the verbosity and reusability

In [21]:
class CotText2SQLPipeline(dspy.Module):
    def __init__(self,predict_mode):
        self.predict_mode = predict_mode
        self.query_generator = dspy.Predict(BasicText2SQL)
        self.generate_query_with_cot = dspy.ChainOfThought(BasicText2SQL)

    def forward(self, question):
        if self.predict_mode == "predict":
            return self.query_generator(question=question)
        else:
            return self.generate_query_with_cot(question=question)

In [22]:
cot = CotText2SQLPipeline(predict_mode="predict")

cot(question=test_dataset[2].question)

Prediction(
    answer='SELECT SUM(Gold) AS Total_Gold_Medals\nFROM table_name\nWHERE Rank = 2 AND Silver < 2'
)

Let's load a metric which we'll use later

In [40]:
exact_match_metric = dspy.evaluate.answer_exact_match

In [23]:
train_dataset_sample = train_dataset[:20]

## TelePrompters

Teleprompters perform optimization to improve the prompt which in turn improves the perfomance of the module. <br>

### LabeledFewShot TelePrompt
LabeledFewShot is the most vanilla Teleprompt which takes training data as an imput and it add a subset in the LLM's demo attribute which essentially means it adds few shot examples to the LLM

In [43]:
from dspy.teleprompt import LabeledFewShot

teleprompter = LabeledFewShot()

compiled_prompt_opt = teleprompter.compile(cot, trainset=train_dataset_sample)

In [44]:
compiled_prompt_opt(question=test_dataset[0].question)

Prediction(
    answer='SELECT Money Spent, 3Q FROM table WHERE Candidate = Dennis Kucinich'
)

In [46]:
print("Question : ",test_dataset[0].question)
print("Actual Answer : ",test_dataset[0].answer)
print("Predicted Answer : ",compiled_prompt_opt(question=test_dataset[0].question))

Question :  question : How much money has Candidate Dennis Kucinich spent? 
 Table Headers : Candidate,Money Raised, 3Q,Loans Received, 3Q,Money Spent, 3Q,Total Receipts,Cash On Hand,After Debt 
 SQL query : 
Actual Answer :  SELECT Money Spent, 3Q FROM table WHERE Candidate = dennis kucinich
Predicted Answer :  Prediction(
    answer='SELECT Money Spent, 3Q FROM table WHERE Candidate = Dennis Kucinich'
)


There are also other TelePrompters for which you can refer the doc [here](https://github.com/stanfordnlp/dspy/blob/main/docs/teleprompters.md)

In [47]:
#save the teleprompter
compiled_prompt_opt.save("teleprompter.json")
#load the teleprompter using module object
cot.load("teleprompter.json")